In [1]:
import os
import pandas as pd
import numpy as np
import time
path = os.path.dirname(os.path.realpath('__file__'))
print(path)
rate = pd.read_csv(path + '\\ML\\ratings.csv')
rate.head()

C:\Users\Rowida Nagah


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [2]:
import os
import pandas as pd
import numpy as np
import time
path = os.path.dirname(os.path.realpath('__file__'))
print(path)
movies = pd.read_csv(path + '\\ML\\movies.csv')
print(movies.shape)
movies.head()

C:\Users\Rowida Nagah
(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
"""
C is the mean vote across the whole report
v is the number of votes for the movie;
m is the minimum votes required to be listed in the chart;
R is the average rating of the movie; And
"""
C = rate["movieId"].mean()  # mean for the whole rate
C

19435.2957177992

In [7]:
v = sorted(rate["movieId"])
from itertools import groupby
v =[len(list(group)) for key, group in groupby(v)]
print(len(v))  #v is the number of votes for each movi
num_rate = pd.Series(v)
movies['vote_count'] = pd.DataFrame(num_rate) 
print(movies.shape)
movies.head()

9724
(9742, 4)


,movieId,title,genres,vote_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,110.0
2,3,Grumpier Old Men (1995),Comedy|Romance,52.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
4,5,Father of the Bride Part II (1995),Comedy,49.0


In [11]:
data = pd.merge_asof(rate , movies, left_index=True, right_index=True)
print(data.shape    ,  rate.shape  , movies.shape)
data.head()

(100836, 8) (100836, 4) (9742, 4)


,userId,movieId_x,rating,timestamp,movieId_y,title,genres,vote_count
0,1,1,4.0,964982703,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
1,1,3,4.0,964981247,2,Jumanji (1995),Adventure|Children|Fantasy,110.0
2,1,6,4.0,964982224,3,Grumpier Old Men (1995),Comedy|Romance,52.0
3,1,47,5.0,964983815,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
4,1,50,5.0,964982931,5,Father of the Bride Part II (1995),Comedy,49.0


In [20]:
r = data.groupby('title').rating.head()    #  the average rating ofr the movier
avg = pd.Series(r)
movies['vote_average'] = pd.DataFrame(avg) 
print(movies.shape)
movies.head()

(9742, 5)


,movieId,title,genres,vote_count,vote_average
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,4.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,110.0,4.0
2,3,Grumpier Old Men (1995),Comedy|Romance,52.0,4.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0,5.0
4,5,Father of the Bride Part II (1995),Comedy,49.0,5.0


In [14]:
m= movies['vote_count'].quantile(0.9)
m

27.0

In [21]:
q_movies= movies.copy().loc[movies['vote_count'] >= m]
q_movies.head()

,movieId,title,genres,vote_count,vote_average
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,4.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,110.0,4.0
2,3,Grumpier Old Men (1995),Comedy|Romance,52.0,4.0
4,5,Father of the Bride Part II (1995),Comedy,49.0,5.0
5,6,Heat (1995),Action|Crime|Thriller,102.0,3.0


In [22]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)


In [28]:
q_movies = q_movies.sort_values('score', ascending=False)
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)


,title,vote_count,vote_average,score
1458,Friday the 13th Part VIII: Jason Takes Manhatt...,27.0,5.0,9720.147859
8671,Bill Burr: Why Do I Do This? (2008),27.0,5.0,9720.147859
8668,Carry on Cabby (1963),27.0,5.0,9720.147859
883,Paths of Glory (1957),27.0,5.0,9720.147859
779,Angels in the Outfield (1994),27.0,5.0,9720.147859
1034,Cat People (1982),27.0,5.0,9720.147859
2984,Dungeons & Dragons (2000),27.0,4.5,9719.897859
9415,Manchester by the Sea (2016),27.0,4.5,9719.897859
1661,Rounders (1998),27.0,4.5,9719.897859
4090,Ragtime (1981),27.0,4.0,9719.647859
